In [1]:
import torch

In [12]:
the_model = torch.load('../CBOW_F_FullyTrained.obj')
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

world = ['America', 'city', 'country', 'Asia', 'Europe', 'China']
lifestyle = ['phone', 'shop', 'mall', 'play', 'hobby', 'life', 'food']
academics = ['physics', 'science', 'mathematics', 'chemistry', 'law']
nature = ['grass', 'trees', 'green', 'leaves', 'sun', 'snow']

sentences =[('Delhi', world),
            ('Washington', world),
            ('Paris', world),
            ('party', lifestyle),
            ('India', world),
            ('eat', lifestyle),
            ('swim', lifestyle),
            ('flag', world),
            ('sports', lifestyle),
            ('concept', academics),
            ('experiment', academics),
            ('sing', lifestyle),
            ('Nobel', academics),
            ('photosynthesis', nature),
            ('weather', nature),
            ('mountain', nature),
            ('garden', nature),
            ('birds', nature),
            ('lawn', nature),
            ('noodles', lifestyle),
            ('dinner', lifestyle),
            ('study', academics),
            ('university', academics),
            ('driving', lifestyle)]
            #('scientist', academics)]



In [13]:
writer = SummaryWriter('runs/analysis')
word2id = the_model.word2id
u_embeddings = the_model.u_embeddings

emo_strings = world + lifestyle + academics + nature
emo_strings = [w for w in emo_strings if w in word2id]
emotions = torch.tensor([word2id[w] for w in emo_strings])
emotions = u_embeddings(emotions)

correct = 0
emo_strings_score = [[0, 0] for w in emo_strings]

for i, sen in enumerate(sentences):
    sen_arr = sen[0].split()
    sen_arr = torch.tensor([word2id[w] for w in sen_arr if w in word2id])
    sen_arr = u_embeddings(sen_arr).squeeze()
    #sen_arr = torch.sum(sen_arr, dim=0)

    #score = F.logsigmoid(torch.matmul(emotions, sen_arr.unsqueeze(1)))
    #score = torch.cdist(emotions, sen_arr.unsqueeze(0)).squeeze()

    score = torch.stack([torch.dot(u,sen_arr)/(torch.norm(u)*torch.norm(sen_arr)) for u in emotions])
    
    pred_emo = emo_strings[torch.argmax(score)] #changed

    print('sentence ', str(i+1), ': ', sen[0])
    print('Predicted subject:', pred_emo)
    print('Actual subject(s):', sen[1], '\n')

    if (pred_emo in sen[1]):
        correct += 1
        emo_strings_score[emo_strings.index(pred_emo)][0] += 1
    else:
        emo_strings_score[emo_strings.index(pred_emo)][1] += 1

correct /= len(sentences)
print('Accuracy:', correct*100, '%')

emo_strings = [ w + ' H ' + str(emo_strings_score[i][0]) + ' M ' + str(emo_strings_score[i][1]) for i,w in enumerate(emo_strings)]

writer.add_embedding(emotions, metadata=emo_strings)
writer.close()
%load_ext tensorboard

emo_strings

sentence  1 :  Delhi
Predicted subject: Asia
Actual subject(s): ['America', 'city', 'country', 'Asia', 'Europe', 'China'] 

sentence  2 :  Washington
Predicted subject: Asia
Actual subject(s): ['America', 'city', 'country', 'Asia', 'Europe', 'China'] 

sentence  3 :  Paris
Predicted subject: Asia
Actual subject(s): ['America', 'city', 'country', 'Asia', 'Europe', 'China'] 

sentence  4 :  party
Predicted subject: Asia
Actual subject(s): ['phone', 'shop', 'mall', 'play', 'hobby', 'life', 'food'] 

sentence  5 :  India
Predicted subject: chemistry
Actual subject(s): ['America', 'city', 'country', 'Asia', 'Europe', 'China'] 

sentence  6 :  eat
Predicted subject: play
Actual subject(s): ['phone', 'shop', 'mall', 'play', 'hobby', 'life', 'food'] 

sentence  7 :  swim
Predicted subject: mall
Actual subject(s): ['phone', 'shop', 'mall', 'play', 'hobby', 'life', 'food'] 

sentence  8 :  flag
Predicted subject: shop
Actual subject(s): ['America', 'city', 'country', 'Asia', 'Europe', 'China'] 


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)

In [7]:
%reload_ext tensorboard